In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('fridge/fridge.csv')
df

,price,title,added_time,metro,seller_name,seller_rating,address,description,dist_to_metro,seller_review,seller_status,views
0,6000,Холодильник,вчера в 10:09,Парк Победы,макс,5.0,"Санкт-Петербург, Кузнецовская ул., 21",✓ Гарантируем лучшие условия в городе на покуп...,500 м,8.0,Частное лицо,353.0
1,6000,Холодильник LG no frost,сегодня в 01:35,NaN,Александр,5.0,"Санкт-Петербург, Пушкинский р-н, пос. Шушары, ...",Холодильник двухкамерный LG GC-309 в хорошем р...,NaN,3.0,Частное лицо,48.0
2,2800,Переносной холодильник новый,сегодня в 00:10,Приморская,Роман,5.0,"Санкт-Петербург, Морская наб., 35к2","Переносной холодильник. Абсолютно новый, в кор...","1,4 км",6.0,Частное лицо,622.0
3,1500,Холодильник Норд,вчера в 23:19,Проспект Ветеранов,Олег,NaN,"Санкт-Петербург, садовые участки",Продам полностью рабочий холодильник Норд дх 4...,"2,5 км",0.0,Частное лицо,78.0
4,5000,Холодильник zanussi,вчера в 23:14,Елизаровская,Александр,5.0,"Санкт-Петербург, ул. Ольги Берггольц, 18","Состояние 4, полностью рабочийТорг",300 м,1.0,Частное лицо,108.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1335,800,Корпус (ящика+панель(щиток) Atlant атлант,1 марта в 21:52,NaN,Павел,NaN,"Ленинградская область, Ломоносовский р-н, Вилл...",в хорошем состоянии. использовалось не долго К...,NaN,0.0,Частное лицо,65.0
1336,6500,Холодильник Атлант хм-6001,5 марта в 09:48,Проспект Ветеранов,Денис,5.0,"Санкт-Петербург, ул. Генерала Симоняка, 8к2",Продам холодильник Атлант ХМ-60012 независимых...,"2,7 км",5.0,Частное лицо,179.0
1337,5500,Холодильник Beko-DSK251,4 марта в 21:38,NaN,Нина,NaN,"Санкт-Петербург, ул. Пограничника Гарькавого",Состояние на 4ку.Перед тем как выставить на пр...,NaN,0.0,Частное лицо,62.0
1338,1270,Ручка Либхер (Liebherr ) Длина 31см (9086742),2 марта в 17:48,Улица Дыбенко,Игорь,5.0,"Санкт-Петербург, ул. Дыбенко",Оригинальные запчасти для холодильника Либхер:...,700 м,17.0,Частное лицо,14089.0


# Посмотрим на NaN

In [3]:
df.isna().sum()

price              0
title              0
added_time         0
metro            198
seller_name        0
seller_rating    822
address            0
description        0
dist_to_metro    425
seller_review      0
seller_status      0
views              0
dtype: int64

NaN-ов в важных столбцах нет. Ближайшее метро и расстояние до него можно будет получить через API Яндекса, а на рейтинг продавца можно не обращать особого внимания.

# Избавимся от объявлений, продающих неисправные холодильники или запчасти для них 
Профильтруем датафрейм по списку слов в заголовке. Потом можно будет сделать более продвинутый алгоритм, который будет работать с описанием товара.  
Функция будет принимать текстовый файл со списком запрещенных слов.

In [4]:
def filter_df(df, filter_file):
    with open(filter_file, 'r', encoding='utf-8') as f:
        filter_words = f.readlines()
    return df[~df.title.str.lower().str.contains('|'.join(filter_words))]

In [5]:
df = filter_df(df, 'fridge/fridge_filter.txt')
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
price,1332.0,3671.012012,1673.496757,750.0,2100.0,3994.5,5000.00,6500.0
seller_rating,513.0,4.524756,1.195055,1.0,5.0,5.0,5.00,5.0
seller_review,1332.0,2.818318,7.785345,0.0,0.0,0.0,1.00,51.0
views,1332.0,476.337087,978.393179,16.0,117.0,238.0,477.25,14733.0


# Переведем строки в более удобные форматы
## Переведем столбец *dist_to_metro*

In [6]:
def dist_to_float(x):
    if type(x) != str:
        return x
    dist, unit = x.split()
    dist = dist.replace(',', '.')
    dist = float(dist)
    if unit == 'км':
        dist *= 1000
    return dist
df['dist_to_metro'] = df.dist_to_metro.map(dist_to_float)

C:\Users\xopclabs\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [7]:
df.dist_to_metro.describe()

count      913.000000
mean      1180.722892
std       1033.517628
min        100.000000
25%        500.000000
50%        900.000000
75%       1500.000000
max      10200.000000
Name: dist_to_metro, dtype: float64

## Переведем столбец *added_time*

In [8]:
df.added_time.str.contains(' в ').value_counts()

True    1332
Name: added_time, dtype: int64

Каждая запись имеет ```' в '``` внутри. Значит, мы сможем безопасно разделить стринг по этому токену

In [9]:
from datetime import datetime, timedelta

def str_to_datetime(x):
    date, time = x.split(' в ')
    hour, minute = list(map(int, time.split(':')))
    if date == 'вчера':
        yesterday = datetime.now() - timedelta(days=1)
        day = yesterday.day
        month = yesterday.month
    elif date == 'сегодня':
        now = datetime.now()
        day = now.day
        month = now.month
    else:
        ru_to_num = {
            'января':1,
            'февраля':2,
            'марта':3,
            'апреля':4,
            'мая':5,
            'июня':6,
            'июля':7,
            'августа':8,
            'сентября':9,
            'октября':10,
            'ноября':11,
            'декабря':12,
        }
        day, month = date.split()
        day = int(day)
        month = ru_to_num[month]
    return datetime(2020, month, day, hour, minute) 

In [10]:
df['added_time'] = df.added_time.map(str_to_datetime)

C:\Users\xopclabs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
df.added_time

0      2020-03-30 10:09:00
1      2020-03-31 01:35:00
2      2020-03-31 00:10:00
3      2020-03-30 23:19:00
4      2020-03-30 23:14:00
               ...        
1335   2020-03-01 21:52:00
1336   2020-03-05 09:48:00
1337   2020-03-04 21:38:00
1338   2020-03-02 17:48:00
1339   2020-03-01 22:57:00
Name: added_time, Length: 1332, dtype: datetime64[ns]

# Получим координаты места по адресу через Геокодер Яндекса

In [12]:
import requests
import json
from secret import API_KEY
example = df.iloc[2].address
example

'Санкт-Петербург, Морская наб., 35к2'

In [13]:
request = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode={}'.format(API_KEY, example.replace(' ', '+'))
r = requests.get(request)
response = json.loads(r.text)
response

{'response': {'GeoObjectCollection': {'metaDataProperty': {'GeocoderResponseMetaData': {'request': 'Санкт-Петербург, Морская наб., 35к2',
     'results': '10',
     'found': '1'}},
   'featureMember': [{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'exact',
        'text': 'Россия, Санкт-Петербург, Морская набережная, 35к2',
        'kind': 'house',
        'Address': {'country_code': 'RU',
         'formatted': 'Россия, Санкт-Петербург, Морская набережная, 35к2',
         'postal_code': '199155',
         'Components': [{'kind': 'country', 'name': 'Россия'},
          {'kind': 'province', 'name': 'Северо-Западный федеральный округ'},
          {'kind': 'province', 'name': 'Санкт-Петербург'},
          {'kind': 'locality', 'name': 'Санкт-Петербург'},
          {'kind': 'street', 'name': 'Морская набережная'},
          {'kind': 'house', 'name': '35к2'}]},
        'AddressDetails': {'Country': {'AddressLine': 'Россия, Санкт-Петербург, Морская набережная, 35к2',
  

In [14]:
def get_coords(x):
    request = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode={}'.format(API_KEY, x.replace(' ', '+'))
    r = requests.get(request)
    coords = json.loads(r.text)['response']
    coords = coords['GeoObjectCollection']['featureMember'][0]
    coords = coords['GeoObject']['Point']['pos']
    coords = coords.split()[::-1]
    coords = list(map(float, coords))
    return coords

In [15]:
get_coords(example)

[59.959138, 30.220359]

In [16]:
df['coords'] = df.address.map(get_coords)

"Растащим" список на две отдельные колонки

In [17]:
df[['lat', 'long']] = pd.DataFrame(df.coords.values.tolist(), index=df.index)
df = df.drop('coords', 1)

Соберем это в отдельную функцию

In [18]:
def geocode(df):
    from secrect import API_KEY
    def get_coords(x):
        request = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode={}'.format(API_KEY, x.replace(' ', '+'))
        r = requests.get(request)
        coords = json.loads(r.text)['response']
        coords = coords['GeoObjectCollection']['featureMember'][0]
        coords = coords['GeoObject']['Point']['pos']
        coords = coords.split()[::-1]
        coords = list(map(float, coords))
        return coords
    
    df['coords'] = df.address.map(get_coords)
    df[['lat', 'long']] = pd.DataFrame(df.coords.values.tolist(), index=df.index)
    df = df.drop('coords', 1)
    

In [19]:
df.to_csv('fridge/fridge_.csv', index=False)